<a href="https://colab.research.google.com/github/chamithZ/DL-Lab-5/blob/main/IT21156960_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import re

In [2]:

# 1. Load and Preprocess the Dataset
def load_data(file_path):
    # Load the dataset (e.g., IMDB movie reviews dataset)
    df = pd.read_csv(file_path, engine='python', on_bad_lines='skip')  # Using 'python' engine and skipping bad lines
    df.dropna(inplace=True)  # Drop any rows with missing values
    return df['review'], df['sentiment']  # Assuming 'review' and 'sentiment' columns

In [3]:
# Clean the text
def clean_text(text):
    # Remove unwanted characters, numbers, and symbols
    text = re.sub(r"[^A-Za-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [4]:

# Tokenize and Pad Sequences
def preprocess_text(reviews, max_words=5000, max_len=200):
    reviews = [clean_text(review) for review in reviews]  # Clean the reviews
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(reviews)
    sequences = tokenizer.texts_to_sequences(reviews)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer

In [5]:
# Encode Sentiments
def encode_labels(sentiments):
    sentiments = sentiments.map({'positive': 1, 'negative': 0}).values
    return sentiments


In [7]:

# Load Data
file_path = '/content/IMDB Dataset.csv'  # <-- Provide the correct path to the dataset
reviews, sentiments = load_data(file_path)

In [8]:

# Preprocess Text Data
max_words = 6000  # Consider the top 5000 words
max_len = 200  # Pad or truncate reviews to 200 words
X, tokenizer = preprocess_text(reviews, max_words=max_words, max_len=max_len)

In [9]:
# Encode Sentiments (positive -> 1, negative -> 0)
y = encode_labels(sentiments)

# Split into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# 2. Define the LSTM Model
model = Sequential()

# Modify the embedding dimensions and experiment with LSTM configurations ---
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))  # <-- Modify 'output_dim'
model.add(Bidirectional(LSTM(units=74, return_sequences=False)))  # <-- Experiment with 'units' and add Dropout if necessary

model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Train the Model
#  Modify 'epochs' and 'batch_size' to see how they impact training time and model accuracy ---
model.fit(X_train, y_train, epochs=5, batch_size=52, validation_data=(X_test, y_test), verbose=1)  # <-- Experiment with 'epochs' and 'batch_size'


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 504s 618ms/step - accuracy: 0.7582 - loss: 0.4825 - val_accuracy: 0.8671 - val_loss: 0.3428
Epoch 2/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 477s 588ms/step - accuracy: 0.8886 - loss: 0.2821 - val_accuracy: 0.8777 - val_loss: 0.3014
Epoch 3/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 502s 587ms/step - accuracy: 0.9196 - loss: 0.2095 - val_accuracy: 0.8879 - val_loss: 0.2761
Epoch 4/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 502s 587ms/step - accuracy: 0.9364 - loss: 0.1707 - val_accuracy: 0.8877 - val_loss: 0.2857
Epoch 5/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 515s 604ms/step - accuracy: 0.9488 - loss: 0.1408 - val_accuracy: 0.8565 - val_loss: 0.3572


In [ ]:
model_unidirectional = Sequential()
model_unidirectional.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_unidirectional.add(LSTM(units=74, return_sequences=False))
model_unidirectional.add(Dropout(0.5))  # Optional, for regularization
model_unidirectional.add(Dense(1, activation='sigmoid'))

model_unidirectional.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the unidirectional model
model_unidirectional.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=1)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


653/653 ━━━━━━━━━━━━━━━━━━━━ 256s 388ms/step - accuracy: 0.7507 - loss: 0.4903 - val_accuracy: 0.8812 - val_loss: 0.2899
Epoch 2/10
 55/653 ━━━━━━━━━━━━━━━━━━━━ 3:18 333ms/step - accuracy: 0.9043 - loss: 0.2360

In [11]:
# 4. Evaluate the Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

327/327 ━━━━━━━━━━━━━━━━━━━━ 27s 81ms/step


In [12]:

# Calculate Accuracy and F1-Score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')

Accuracy: 0.8565
F1-Score: 0.8657


In [ ]:
y_pred_2 = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:

# Calculate Accuracy and F1-Score
accuracy = accuracy_score(y_test, y_pred_2)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')